I dette projekt vil vi gøre det muligt, at forudsige en karakter fra 1-5 ud fra tekstindholdet i en anmeldelse på Trustpilot,
og vise om sammenhængen af én virksomheds samlede vurdering passer mellem karakter og anmeldelser.
Vi har tænkt os at gøre dette ved at scrape en masse andmeldelser fra forskellige virksomheder på Truspilot ved brug af selenium. Vi vil basere vores model ud fra de indsamlede data.

Med vores model vil vi:

1. Evaluére og fastslå hvor brugbar modellen er ved at sammenligne træningsdata- og valideringsdata.
<br>1a. Efterjustér modellens definition efter behov.


2. Forudsige hvilken karakter en anmelder vil give ud fra den anmeldselse de har skrevet. (Regression)
<br>2a. Sammenlign med den aktuelle anmelder karakter.


3. Vis sansynligheden for predictions indenfor Trustpilots fem katagorier: Fremragende, God, Middel, Uder middel, Dårlig, for en specifik anmeldelse. (Classification)


4. Gruppere og plot ratings for én specifik virksomhed i et 3D feature space. (Clustering)


5. Brug WordCloud til og plotte de mest anvendte Negative og Positive ord.

In [7]:
#%pip install requests==2.27.1

import bs4
import csv
import requests
import re
from concurrent.futures import ThreadPoolExecutor

#%matplotlib notebook

print('bs4\t\t', bs4.__version__)
print('csv\t\t', csv.__version__)
print('requests\t', requests.__version__)
print('re\t\t', re.__version__)

bs4		 4.8.2
csv		 1.0
requests	 2.23.0
re		 2.2.1


In [8]:
allShopsRequest = []

initUrl1 = "https://dk.trustpilot.com/review/www.proshop.dk"
initUrl2 = "https://dk.trustpilot.com/review/zalando.dk"

allShopsRequest.append(initUrl1)
allShopsRequest.append(initUrl2)

print("OK")


OK


In [9]:
allPagesRequest = []

def multiThreadDownload(shop):
    listofpages=""
    totalpages=None
    
    req = requests.get(shop, timeout=10)
    soup = bs4.BeautifulSoup(req.content, 'html.parser')
    
    try:      
        totalpages = soup.find('a', attrs={'name':'pagination-button-last'}).text
    except:       
        listofpages = [7,6,5,4,3,2,1]
        try:
            for content in listofpages:
                totalpages = soup.find('a', attrs={'name':'pagination-button-'+str(content)}).text
                if totalpages is not None:
                    print('Amount of review pages found:', totalpages)
                    break
                
        except Exception as e:
            print('Error', e)
    
    for i in range(int(totalpages)):
        i+=1
        url = shop+'?page='+str(i)
        allPagesRequest.append(requests.get(url, timeout=10))

with ThreadPoolExecutor(4) as ex:
    ex.map(multiThreadDownload, allShopsRequest)


KeyboardInterrupt: 

In [10]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

print("OK")

OK


In [11]:
reviewErrorCount = [];

def multithreadwritecsvfile(page):
    with open('output.csv', 'a', newline='', encoding='utf-8') as output_file:
        output_writer = csv.writer(output_file)
        soup = bs4.BeautifulSoup(page.content,'html.parser')
        for content in soup.find_all('section', attrs={'class':'styles_reviewContentwrapper__zH_9M'}):
            rating=None
            splitted=None
            title=None
            review=None
            
            try: 
                rating = content.find('img',alt = True)
                splitted = rating.get('alt').split()
            except Exception as e:
                print('No rating found. Skipping...', e)
            
            try: 
                title = content.find('a', attrs={'name':'review-title'}).text
                title = remove_emojis(title)
            except Exception as e:
                print('No title found. Skipping...', e)
                
            try: 
                review = content.find('p', attrs={'class':'typography_typography__QgicV typography_body__9UBeQ typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3'}).text
                if len(review) > 2:
                    review = remove_emojis(review)
                    review = review.replace(",", " ")
                    review = review.replace(' "" ', "")
                    output_writer.writerow(['__label__'+splitted[2]+" ", review])
            except Exception as e:
                #print('No review found. Skipping...', e)
                reviewErrorCount.append(e);
            
with ThreadPoolExecutor(8) as ex:
    with open('output.csv', 'w', newline='', encoding='utf-8') as output_file:
        output_writer = csv.writer(output_file)
        output_writer.writerow(['rating','review'])
    ex.map(multithreadwritecsvfile, allPagesRequest)

print('Done. Skipped reviews:', len(reviewErrorCount))

Done. Skipped reviews: 480
